## Using a benchmark problem
This tutorial shows how to use and query a benchmark problem.

In [1]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

The following loads a benchmark problem, from which we get the ry.MathematicalProgram interface

In [2]:
#benchmark = ry.OptBenchmark_InvKin_Endeff("../rai/test/KOMO/switches/model2.g", True)
benchmark = ry.OptBench_Skeleton_Handover(ry.Arg.path)
# benchmark = ry.OptBench_Skeleton_StackAndBalance(ry.Arg.sequence)
nlp = benchmark.get()

In [3]:
help(nlp)

Help on MathematicalProgram in module libry object:

class MathematicalProgram(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      MathematicalProgram
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  evaluate(...)
 |      evaluate(self: libry.MathematicalProgram, arg0: rai::Array<T>) -> Tuple[rai::Array<T>, rai::Array<T>]
 |      
 |      query the NLP at a point $x$; returns the tuple $(phi,J)$, which is the feature vector and its Jacobian; features define cost terms, sum-of-square (sos) terms, inequalities, and equalities depending on 'getFeatureTypes'
 |  
 |  getBounds(...)
 |      getBounds(self: libry.MathematicalProgram) -> Tuple[rai::Array<T>, rai::Array<T>]
 |      
 |      returns the tuple $(b_{lo},b_{up})$, where both vectors are of same dimensionality of $x$ (or size zero, if there ar

The first method to use is `getInitializationSample`, which returns an initial x

In [4]:
x = nlp.getInitializationSample()

The most important method is `evaluate(x)`, which returns a tuple `(phi,J)`

In [ ]:
(phi,J) = nlp.evaluate(x)
print('phi=', phi, '\nJ=', J)

In [22]:
#note the dimensionalities:
print('dim(x)=', nlp.getDimension())
print('dim(x)=', x.shape)
print('dim(phi)=', phi.shape)
print('dim(J)=', J.shape)

dim(x)= 239
dim(x)= (239,)
dim(phi)= (331,)
dim(J)= (331, 239)


The features `phi` define all cost terms and inequality/equality constraint values. The feature types tell you, which feature entries refer to costs/constraints. For each feature, an integer is returned -- but actually it is an enum. So you can test equality with ry.OT.f, ry.OT.sos, ry.OT.ineq, or ry.OT.eq

In [23]:
types = nlp.getFeatureTypes()
print(types)
print(types[0] == ry.OT.f)
print(types[0] == ry.OT.sos)
print(types[0] == ry.OT.ineq)
print(types[0] == ry.OT.eq)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 2 2 2 2 2 2 2 2 2
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 3 3 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 2 2]
False
True
False
False


So, in our case, all features are sum-of-square features, the total cost function is $\sum_i \phi_i^2$, and we have no constraints.

Finally, the benchmark problem implement a report, which should help you to get more insight in what $x$ means and what is computed. For high verbosity it also displays something - in our case the robot in pose $x$.

In [24]:
# the report method generally returns some info string - internal information given by the benchmark implementation
print(nlp.report(4))

KOMO Problem:
  x-dim:239  dual-dim:0
  T:6 k:1 phases:6 stepsPerPhase:1 tau:2
  #timeSlices:7 #totalDOFs:239 #frames:392
    times:[2, 4, 6, 8, 10, 12]
  computeCollisions:0
    TASK 'F_qItself/1-#28' (-1..5)  type:sos  order:1  target:[]  scale:[0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421]
    TASK 'F_qQuaternionNorms/0-#56' (0..5)  type:eq  order:0  target:[]  scale:[3]
    TASK 'F_Pose/1-box0' (-1..0)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_PoseRel/1-box0-R_endeff' (0..1)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_qZeroVel/1-box0' (1..5)  type:eq  order:1  target:[]  scale:[10]
    TASK 'F_qZeroVel/1-box1' ()  type:eq  order:1  target:[]  scale:[10]
    TASK 'F_Pose/1-box1' (0..1)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_PoseRel/1-box1-L_endeff' (1..2)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_qZeroVel/1-box1' (2..5)  type:eq  order:1  tar

## demo for testing
Calling the rai optimizer as below is just for testing - you're supposed to instead implement your own solvers!

In [25]:
#setup a solver with that problem
solver = ry.NLP_Solver()
solver.setProblem(nlp)
#solver.setSolver(ry.NLP_SolverID.newton)
solver.setSolver(ry.NLP_SolverID.augmentedLag)

In [26]:
#solve it
solver.solve(True)

array([ -4.80260966e-02,   5.68230854e-02,  -4.77892129e-02,
        -9.25782228e-01,  -3.49027995e-01,   1.38815096e-01,
         4.88494601e-02,  -8.26641542e-03,  -3.15379013e-01,
        -2.94562374e-02,  -1.28957006e+00,  -6.28283526e-03,
        -1.29285368e+00,   1.44414872e-02,   4.00427962e-01,
        -7.22610173e-01,   2.98734935e-01,  -1.46884096e+00,
         4.33346522e-01,  -1.44458388e+00,   4.06495973e-01,
        -1.41584075e-01,   4.34691849e-02,   1.09718292e-01,
        -7.71154626e-01,  -5.03488785e-01,  -2.11752516e-01,
        -3.27765566e-01,   1.66874518e-01,   5.35813806e-02,
        -8.99987806e-02,  -9.42388061e-01,  -9.34621014e-02,
        -1.14465833e-01,  -3.00399795e-01,  -2.57798293e-01,
        -7.00669883e-01,  -2.52680710e-01,  -1.21873796e+00,
        -9.48538684e-02,  -1.39034128e+00,   1.76301941e-01,
         3.64864121e-01,  -7.41271136e-01,   4.14669009e-01,
        -1.33270911e+00,   1.81073085e-02,  -7.41830395e-01,
        -4.26185681e-01,

In [27]:
solver.getTrace_x()

array([[ -4.97831591e-01,  -4.78137445e-01,  -1.50663069e-01, ...,
          2.12807887e-02,   9.02864704e-03,  -9.57494335e-03],
       [ -4.82680681e-01,  -4.67712644e-01,  -1.67444410e-01, ...,
          2.10583491e-02,   8.86721985e-03,  -2.88747077e-02],
       [ -4.55883452e-01,  -4.49123628e-01,  -1.73056256e-01, ...,
          2.00010031e-02,   7.21924317e-03,  -7.09936426e-02],
       ..., 
       [ -4.80260709e-02,   5.68230637e-02,  -4.77892712e-02, ...,
         -3.16372595e-07,  -7.39173629e-06,  -9.99998233e-01],
       [ -4.80260812e-02,   5.68230724e-02,  -4.77892479e-02, ...,
         -3.16372534e-07,  -7.39173368e-06,  -9.99998233e-01],
       [ -4.80260966e-02,   5.68230854e-02,  -4.77892129e-02, ...,
         -3.16372443e-07,  -7.39172975e-06,  -9.99998233e-01]])

In [28]:
#the first is total cost, 2nd is ineq, 3rd is eq
solver.getTrace_costs()

array([[  1.44578254e-04,   6.81067818e+00,   9.58271848e+02],
       [  1.83081491e-03,   5.06009148e+00,   9.50332756e+02],
       [  8.39637747e-03,   5.80959711e+00,   9.32980485e+02],
       [  9.31484652e-03,   0.00000000e+00,   8.80556115e+02],
       [  1.42554381e-02,   0.00000000e+00,   8.24491330e+02],
       [  1.58414246e-02,   0.00000000e+00,   7.60520827e+02],
       [  1.74896422e-02,   0.00000000e+00,   6.94497215e+02],
       [  1.88032638e-02,   0.00000000e+00,   6.27856641e+02],
       [  2.09652325e-02,   0.00000000e+00,   5.60947159e+02],
       [  2.07568892e-02,   0.00000000e+00,   4.93776251e+02],
       [  2.21019925e-02,   0.00000000e+00,   4.26153494e+02],
       [  2.34033852e-02,   0.00000000e+00,   3.58216669e+02],
       [  2.56825109e-02,   0.00000000e+00,   2.88993770e+02],
       [  2.85167699e-02,   0.00000000e+00,   2.19831884e+02],
       [  3.27098806e-02,   0.00000000e+00,   1.51556983e+02],
       [  3.87935043e-02,   0.00000000e+00,   8.2874865

In [29]:
print(nlp.report(4))

KOMO Problem:
  x-dim:239  dual-dim:0
  T:6 k:1 phases:6 stepsPerPhase:1 tau:2
  #timeSlices:7 #totalDOFs:239 #frames:392
    times:[2, 4, 6, 8, 10, 12]
  computeCollisions:0
    TASK 'F_qItself/1-#28' (-1..5)  type:sos  order:1  target:[]  scale:[0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421]
    TASK 'F_qQuaternionNorms/0-#56' (0..5)  type:eq  order:0  target:[]  scale:[3]
    TASK 'F_Pose/1-box0' (-1..0)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_PoseRel/1-box0-R_endeff' (0..1)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_qZeroVel/1-box0' (1..5)  type:eq  order:1  target:[]  scale:[10]
    TASK 'F_qZeroVel/1-box1' ()  type:eq  order:1  target:[]  scale:[10]
    TASK 'F_Pose/1-box1' (0..1)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_PoseRel/1-box1-L_endeff' (1..2)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_qZeroVel/1-box1' (2..5)  type:eq  order:1  tar

In [30]:
print(nlp.report(6))

KOMO Problem:
  x-dim:239  dual-dim:0
  T:6 k:1 phases:6 stepsPerPhase:1 tau:2
  #timeSlices:7 #totalDOFs:239 #frames:392
    times:[2, 4, 6, 8, 10, 12]
  computeCollisions:0
    TASK 'F_qItself/1-#28' (-1..5)  type:sos  order:1  target:[]  scale:[0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421, 0.141421]
    TASK 'F_qQuaternionNorms/0-#56' (0..5)  type:eq  order:0  target:[]  scale:[3]
    TASK 'F_Pose/1-box0' (-1..0)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_PoseRel/1-box0-R_endeff' (0..1)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_qZeroVel/1-box0' (1..5)  type:eq  order:1  target:[]  scale:[10]
    TASK 'F_qZeroVel/1-box1' ()  type:eq  order:1  target:[]  scale:[10]
    TASK 'F_Pose/1-box1' (0..1)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_PoseRel/1-box1-L_endeff' (1..2)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_qZeroVel/1-box1' (2..5)  type:eq  order:1  tar

In [17]:
nlp = 0
benchmark = 0
solver = 0